In [37]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

In [38]:
import numpy as np
from tqdm.notebook import trange
from tqdm.notebook import tqdm

import model
import model_parameters as MP

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [39]:
C = 10 # number of campaigns
U = 1000 # number of customers.
H = 3 # number of channels.
D = 7 # number of planning days.
I = 3 # number of quota categories.
P = 10 # number of priority categories.

In [40]:
print(f"number of campaigns {C}") #70 -camp.
print(f"number of customers {U}") #20.000.000 non-distinct
print(f"number of channels {H}") #6-7 h. --Drop
print(f"number of planning days {D}")
print(f"number of quota categories {I}")
print(f"number of priority categories {P}")

number of campaigns 10
number of customers 1000
number of channels 3
number of planning days 7
number of quota categories 3
number of priority categories 10


#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [43]:
e_cu = np.random.choice(2,(C, U)) #e_cu = np.ones((C, U), dtype='int8')
e_cu

array([[0, 0, 1, ..., 1, 0, 0],
       [1, 1, 0, ..., 0, 1, 1],
       [0, 1, 1, ..., 1, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 1, 1],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 1]])

##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [44]:
q_ic = np.random.choice(2, (I,C)) #q_ic = np.zeros((I,C), dtype='int8')
q_ic

array([[1, 1, 0, 1, 1, 1, 1, 1, 0, 0],
       [1, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [45]:
r_p = np.random.choice(100, P) #r_p = np.ones(P, dtype='int8')
rp_c = np.array([r_p[r] for r in np.random.choice(P, C)])

rp_c

array([19, 11, 17, 17,  0, 19, 35, 11, 97, 43])

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [46]:
b = 7
b

7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [47]:
k = 3
k

3

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [48]:
l_c = np.random.choice([2,3,4],C)
l_c

array([4, 3, 4, 4, 3, 2, 4, 4, 3, 3])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [49]:
m_i = np.random.choice([4,3,5],I)#m_i = np.ones((I), dtype='int8')*10
n_i = np.random.choice([1,3,2],I)#n_i = np.ones((I), dtype='int8')*10
(m_i, n_i)

(array([5, 4, 3]), array([3, 3, 1]))

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [50]:
t_hd = np.random.choice([U*.7, U*.6, U*.5], (H, D))
t_hd

array([[700., 700., 700., 500., 600., 500., 500.],
       [600., 700., 500., 700., 600., 500., 500.],
       [700., 500., 700., 600., 500., 700., 700.]])

# Model

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [12]:
X_cuhd = np.zeros((MP.C,MP.U,MP.H,MP.D), dtype='int')

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall h \in H,\forall d \in D
$$

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

In [13]:
eligibility = lambda X, c, u, h, d: X[c,u,h,d]<=e_cu[c,u]
weekly_limitation = lambda X, u: X[:,u,:,:].sum() <= b
daily_limitation = lambda X, u, d: X[:,u,:,d].sum() <= k
campaign_limitation = lambda X, c, u: X[c,u,:,:].sum() <= l_c[c]
weekly_quota = lambda X, u: all((q_ic * X[:,u,:,:].sum(axis=(1,2))).sum(axis=1)<=m_i)
daily_quota = lambda X, u, d: all((q_ic * X[:,u,:,d].sum(axis=(1))).sum(axis=1)<=n_i)
channel_capacity = lambda X, h, d: X[:,:,h,d].sum() <= t_hd[h,d]

<function model_parameters.<lambda>(X, c, u, h, d)>

In [21]:
#eligibility = lambda X, c, u, h, d: X[c,u,h,d]<=e_cu[c,u]
#channel_capacity = lambda X, h, d: X[:,:,h,d].sum() <= t_hd[h,d]
#weekly_limitation = lambda X, u: X[:,u,:,:].sum() <= b
#daily_limitation = lambda X, u, d: X[:,u,:,d].sum() <= k
#campaign_limitation = lambda X, c, u: X[c,u,:,:].sum() <= l_c[c]
#one_channel = lambda X, c, u, d: X[c,u,:,d].sum() <= 1
#weekly_quota = lambda X, u: np.all(q_ic[:,c] * X[c,u,:,:].sum()<=m_i)
#daily_quota = lambda X, u, d: np.all(q_ic[:,c] * X[c,u,:,d].sum()<=n_i)

c_i = 0
u_i = 1
h_i = 2
d_i = 3

mdl = model.Model([
    model.Constraint('eligibility',MP.eligibility, (c_i, u_i, h_i, d_i,)),
    model.Constraint('channel_capacity',MP.channel_capacity, (h_i, d_i,)),
    model.Constraint('daily_limitation',MP.daily_limitation, (u_i, d_i,)),
    model.Constraint('weekly_limitation',MP.weekly_limitation, (u_i,)),
    model.Constraint('campaign_limitation',MP.campaign_limitation, (c_i, u_i,)),
    model.Constraint('daily_quota',MP.daily_quota, (u_i, d_i,)),
    model.Constraint('one_channel',MP.one_channel, (c_i, u_i, d_i,)),
    model.Constraint('weekly_quota',MP.weekly_quota, (u_i,))
], MP.objective_fn)

In [56]:
X_vuhd = np.zeros((2,4,6,8), dtype='int')

In [66]:
def mdlExec(*args):
    return True

In [68]:
%%timeit
X_v = np.vectorize(mdlExec)(X_vuhd)

69.9 µs ± 3.38 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [71]:
%%timeit
X_v = [mdlExec(X_vuhd[v,u,h,d]) for v in range(2)  for u in range(4)  for h in range(6)  for d in range(8)]
                

171 µs ± 6.58 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
X_v = [mdlExec(X_vuhd[v,u,h,d]) for v in range(2)  for u in range(4)  for h in range(6)  for d in range(8)]


In [22]:
%%time
X_cuhd = np.zeros((MP.C,MP.U,MP.H,MP.D), dtype='int')

def 
for c in tqdm(np.argsort(-MP.rp_c), desc="Campaigns Loop"):
    for d in trange(MP.D, desc=f"Days Loop for campaign-{c}"):
        for h in range(MP.H):#trange(H, desc=f"Channels Loop at Day-{d}, Campapaign-{c}"):
            for u in range(MP.U):#trange(U, desc=f"Users Loop On Campaign-{c}"):
                if not mdl.execute(X_cuhd, (c, u, h, d)):
                    

Campaigns Loop:   0%|          | 0/10 [00:00<?, ?it/s]

Days Loop for campaign-2:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-9:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-7:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-3:   0%|          | 0/7 [00:00<?, ?it/s]

...

Days Loop for campaign-8:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-1:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-0:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-4:   0%|          | 0/7 [00:00<?, ?it/s]

Wall time: 5.7 s


In [23]:
for s in mdl.stats():
    print(s)

('eligibility', 210000)
('channel_capacity', 105777)
('daily_limitation', 75945)
('weekly_limitation', 75871)
('campaign_limitation', 46647)
('daily_quota', 26622)
('one_channel', 12744)
('weekly_quota', 12406)


In [24]:
display(X_cuhd.sum())
display(mdl.calc_value(X_cuhd))

6776

483213

##### - check criterion

- Sort Campaigns by rp_c
- Map every criteria to function